In [ ]:
#!pip install -r requirements.txt
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
f = "data/tcga_brca_all_clean.csv.zip"

#TODO slide_id -> file path 

## Preprocessing genomic data 

In [ ]:
n_bins = 4
df = pd.read_csv(f,compression='zip')

df_uncensored = (df[df["censorship"]==0]).drop_duplicates(["case_id"])
_,bins = pd.qcut(df_uncensored['survival_months'],q = n_bins,retbins=True)  # distribute censored survival months into quartiles

# adapt bins 
bins[0] = 0 
bins[-1] = np.inf
#
labels=[i for i in range(n_bins)]
labels

df.insert(6,"survival_months_discretized",  pd.cut(df["survival_months"],
                                                               bins=bins, 
                                                               labels=labels)) # insert binned survival momnth 
k = 7
df.insert(3,"kfold",df.index%k) # insert kfold 
df.head(5)

scale genomic data first and store in DF again
then add labels to stratify 

In [ ]:
# todo add new column k-th-fold that contains an int in arange(k) where k is a cluster 
groundtruth = df["survival_months_discretized"]
censorship = df["censorship"]
genomics = df[df.keys()[11:]]
genomics


In [ ]:


scaler = StandardScaler()
scaled_genomics = scaler.fit_transform(genomics)
#scaler.inverse_transform(scaled_genomics)
df_scaled = df.copy()
df_scaled[df.keys()[11:]] = scaled_genomics



###  This leads to the full function:

In [ ]:
def clean2trainable(df_path,k,n_bins=4,savename = None):
    df = pd.read_csv(df_path,compression='zip')

    # get time bins 
    df_uncensored = (df[df["censorship"]==0]).drop_duplicates(["case_id"])
    _,bins = pd.qcut(df_uncensored['survival_months'],q = n_bins,retbins=True)  # distribute censored survival months into quartiles

    # adapt time bins 
    bins[0] = 0 
    bins[-1] = np.inf
    # bin name = index 
    labels = [i for i in range(n_bins)]
    df.insert(6,"survival_months_discretized",  pd.cut(df["survival_months"],
                                                               bins=bins, 
                                                               labels=labels)) # insert binned survival month 
    df.insert(3,"kfold",df.index%k) # insert kfold 

    genomics = df[df.keys()[11:]]

    scaler = StandardScaler()
    scaled_genomics = scaler.fit_transform(genomics)
    df[df.keys()[11:]] = scaled_genomics

    if savename is not None:
        df.to_csv(savename,index=False)
    return df 

In [ ]:
f = "data/tcga_brca_all_clean.csv.zip"
new_df = clean2trainable(f,5,4,savename = "./data/tcga_brca_trainable.csv")
#new_df = clean2trainable(f,7,4)

In [ ]:
f_new = "data/tcga_brca_trainable.csv"
df = pd.read_csv(f_new)
df.head()

In [ ]:
df[df.keys()[11:]].head()

## k-Fold crossvalidation
1. Cluster dataset into k groups (shuffle first(seeded) then stratify by patient,split into equal chunks)
2. use one group for testing and the others for training
3. rotate such that each group is once used for testing
4. use distribution of testing results to estimate real value 



In [ ]:
k = 7 
a = np.arange(k) # df k-th fold 
for i in range(k):  
    a = (a+1)%k
    print(f"train on {a[:-1]} test on {a[-1]}, store results in foldername: experiment1_fold{a[-1]}")



In [ ]:
df = pd.DataFrame({"col1": np.arange(100),"col2":np.random.rand(100)})
df
df["col1"] = df["col1"].apply(lambda x: f"thisis{x}")
df

In [ ]:
f = "data/tcga_brca_all_clean.csv.zip"
f = f.replace("all_clean.csv.zip","trainable.csv")
f

In [ ]:
!pip install torchmetrics


In [ ]:
import torch 
idx = torch.randint(low=0,high=4,size=(100,))
values = torch.zeros(size=(100,4))
source = torch.ones(size=(100,))

In [ ]:
values.scatter_(dim=1,index=idx,src=source)

In [ ]:
groundtruth = torch.zeros(size=(10,4))
a,b = groundtruth.size()
labels = torch.randint(low=0,high=b,size=(1,a))
groundtruth[torch.arange(a),labels]=1
groundtruth

In [ ]:
from torchmetrics import Accuracy
acc = Accuracy("multiclass")

In [ ]:
a = torch.randint(low=0,high=2,size=(100,))
a



In [ ]:

from sksurv.metrics import concordance_index_censored
c = torch.randint(0,2,size=(100,))
l = torch.randint(0,4,size=(100,))
out = nn.Sigmoid()(torch.rand(size=(100,4)))
c_index = concordance_index_censored(c,l,out)


## Transformer 


In [ ]:
import torch 
from torch import nn 

class Classifier_Head(nn.Module):
    def __init__(self,outsize,d_hidden=256,t_bins=4):
        super(Classifier_Head,self).__init__()

        self.linear1 = nn.Linear(outsize,d_hidden)
        torch.nn.init.kaiming_normal_(self.linear1.weight)
        self.activ1 = nn.ReLU()
        self.linear2  = nn.Linear(d_hidden,d_hidden)
        torch.nn.init.kaiming_normal_(self.linear2.weight)
        self.activ2 = nn.ReLU()
        self.fc = nn.Linear(d_hidden,t_bins) # TODO test add layer
    def forward(self,x):
        x = torch.flatten(x,start_dim=1)
        x = self.activ1(self.linear1(x))
        x = self.activ2(self.linear2(x))
        return self.fc(x)

class Attention_surv(nn.Module):
    def __init__(self,hist_dim,bins):
        super(Attention_surv,self).__init__()
        self.mhsa = nn.MultiheadAttention(hist_dim,num_heads=4,dropout=0.3,batch_first=True)
        self.head = Classifier_Head(hist_dim,t_bins = bins)
    def forward(self,hist):
        attn_output, attn_output_weights = self.mhsa(hist,hist,hist)
        attn_output = attn_output.mean(dim=1)
        return self.head(attn_output)



In [ ]:
from multi_modal_ds import HistGen_Dataset
import pandas as pd
from utils import Survival_Loss
alpha = 0.25
bins = 4
lr = 2e-4
l1_lambda = 1e-5

f = f"/nodes/bevog/work4/seibel/data/tcga_brca_trainable{bins}.csv"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_path = "/nodes/bevog/work4/seibel/data/TCGA-BRCA-DX-features/tcga_brca_20x_features/pt_files" # folderpath for h5y fiels which contain the WSI feat vecs 
batchsize = 1  # due to different size of bags 

df = pd.read_csv(f)
train_ds = HistGen_Dataset(df,data_path = data_path,train=True)
test_ds = HistGen_Dataset(df,data_path = data_path,train=False)
training_dataloader = torch.utils.data.DataLoader( train_ds,batch_size=batchsize)
test_dataloader = torch.utils.data.DataLoader(test_ds,batch_size=batchsize)


model = Attention_surv(hist_dim=2048,bins=bins).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=lr,betas=[0.9,0.999],weight_decay=1e-5,)
criterion = Survival_Loss(alpha)


In [ ]:
for idx,(histo,gen,c,l,_) in enumerate(training_dataloader):
    histo = histo.to(device)
    out = model(histo)
    out = out.cpu()
    #weights = model.mhsa.parameters()
    loss = criterion(out,c,l) #+ l1_lambda * torch.norm(weights.cpu(),1)
    loss.backward()
    break

# KaplanMeier


In [ ]:
from sksurv.nonparametric import kaplan_meier_estimator
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
bins = 4
f = f"/nodes/bevog/work4/seibel/data/tcga_brca_trainable{bins}.csv"
df = pd.read_csv(f)
df = df[["slide_id","survival_months_discretized","censorship","survival_months"]]

In [ ]:
#help(kaplan_meier_estimator)
event = np.asarray(1-df["censorship"]).astype(bool)
time_exit = np.asarray(df["survival_months"])


x_full, y_full = kaplan_meier_estimator(event, time_exit)
plt.step(x_full, y_full, where="post")
plt.ylim(0, 1.1)
plt.grid()
plt.show


In [ ]:
from models import AttMil_Survival
from multi_modal_ds import HistGen_Dataset
import torch 
from torch import nn 
from utils import c_index
import numpy as np
import pandas as pd
#load model 
d_hist = 2048
device = 'cuda' if torch.cuda.is_available() else 'cpu'
bins = 4
model = AttMil_Survival(d_hist,bins,device ).to(device)
#load weights 
f_weights = "/work4/seibel/results/alpha0.5hist/AttMil_Survival_nll-alpha0.5-fold5-l1_lambda1e-07.pth"
weights = torch.load(f_weights)['model']
model.load_state_dict(weights)



In [ ]:

f =  f"/nodes/bevog/work4/seibel/data/tcga_brca_trainable{bins}.csv"
df = pd.read_csv(f)
data_path = "/nodes/bevog/work4/seibel/data/TCGA-BRCA-DX-features/tcga_brca_20x_features/pt_files" 
test_ds = HistGen_Dataset(df,data_path,train=False)
test_dataloader = torch.utils.data.DataLoader(test_ds,batch_size=1)

out_all_val =torch.empty(size=(len(test_dataloader),bins),device='cpu')        
l_all_val = torch.empty(size=(len(test_dataloader),),device='cpu').to(torch.int16)
l_cont_all_val = torch.empty(size=(len(test_dataloader),),device='cpu').to(torch.int16)
c_all_val = torch.empty(size=(len(test_dataloader),),device='cpu').to(torch.int16)

model.eval()
with torch.no_grad():
    for  idx,(histo,gen,c,l,l_cont) in enumerate(test_dataloader):
        x = histo.to(device)
        out = model(x)
        out = out.cpu()
        
        out_all_val[idx,:] = out
        l_all_val[idx] = l
        c_all_val[idx] = c
        l_cont_all_val[idx] = l_cont

h = nn.Sigmoid()(out_all_val)
S = torch.cumprod(1-h,dim = -1)
risk = -S.sum(dim=1) ## TODO why is it not 1-S ???




In [ ]:
from utils import c_index
import matplotlib.pyplot as plt

c_ind = c_index(out_all_val,c_all_val,l_all_val)
plt.title(f"risk distribution with c_index= {round(c_ind,3)}")
plt.hist(risk.numpy(),bins = 100)
plt.show()
test_ds.gen_depth()

In [ ]:
rows = 4 
eps=0.1
threshold = np.linspace(risk.min()+eps,risk.max()-eps,num=12)
fig,ax1 = plt.subplots(rows,len(threshold)//rows,figsize=(24,8))
fig.tight_layout()
for i in range(len(threshold)):
    risk_bool = risk>threshold[i]
    event1 = np.asarray(1-c_all_val[risk_bool]).astype(bool)
    time_exit1 = np.asarray(l_cont_all_val[risk_bool])
    x1, y1 = kaplan_meier_estimator(event1, time_exit1)

    event2 = np.asarray(1-c_all_val[~risk_bool]).astype(bool)
    time_exit2 = np.asarray(l_cont_all_val[~risk_bool])
    x2, y2 = kaplan_meier_estimator(event2, time_exit2)
    
    x_full, y_full = kaplan_meier_estimator(event, time_exit)

    ax1.flatten()[i].set_title(f"KM for threshold {round(threshold[i],3)}")
    ax1.flatten()[i].step(x1, y1, where="post",label="high risk")
    ax1.flatten()[i].step(x2, y2, where="post",label="low risk")
    ax1.flatten()[i].step(x_full, y_full, where="post",label="full KM")
    
    ax1.flatten()[i].set_ylim(0, 1.1)
    ax1.flatten()[i].grid()
    ax1.flatten()[i].legend()


#plt.clf
#plt.title(f"KM for threshold {round(threshold[i])}")
#plt.step(x1, y1, where="post",label="high risk")
#plt.step(x2, y2, where="post",label="low risk")
#plt.ylim(0, 1.1)
#plt.grid()
#plt.legend()
#plt.show

In [ ]:
plt.title("Scatter risk vs survival month")
plt.scatter(risk[c_all_val.type(torch.bool)],l_cont_all_val[c_all_val.type(torch.bool)],s = 4,label="censored")
plt.scatter(risk[~(c_all_val.type(torch.bool))],l_cont_all_val[~(c_all_val.type(torch.bool))],s = 4,label="uncensored")
plt.xlabel("Risk score")
plt.ylabel("Survival month")
plt.legend()
plt.grid()

In [ ]:
plt.clf()
plt.title(f"risk distribution")
plt.hist(risk[c_all_val.type(torch.bool)].numpy(),bins = 30,alpha=0.5,label="a")
plt.hist(risk[~(c_all_val.type(torch.bool))].numpy(),bins = 30,alpha=0.5,label="b")
plt.legend()
plt.show()

In [ ]:
import wandb
import matplotlib.pyplot as plt 
wandb.init(project="MultiModal",entity="tobias-seibel",name="tests")
out_all_val = out_all_val
n_thresholds = 4
nbins = 30
c_all_val = c_all_val
l_cont_all_val = l_cont_all_val 

#risk
def get_risk(out):
    h = nn.Sigmoid()(out)
    S = torch.cumprod(1-h,dim = -1)
    risk = -S.sum(dim=1)
    return risk
risk_all = get_risk(out_all_val)

#thresholds
min,max = risk_all.min(),risk_all.max()
thresholds = torch.linspace(min,max,n_thresholds+2)[1:-1]

#hist 
censored = c_all_val.type(torch.bool)
uncensored = ~c_all_val.type(torch.bool)

x_c1 = torch.histc(risk_all[censored],bins=nbins,min = min , max =max ) 
x1_label = torch.ones_like(x_c1)
x_c2 = torch.histc(risk_all[uncensored],bins=nbins,min = min , max =max ) 
x2_label = torch.zeros_like(x_c2)
x_all = torch.stack((x_c1,x_c2)).T


#table = wandb.Table(data = x_all.tolist(),columns=["censored","uncensored"])
table = wandb.plot.line_series(
          xs =np.linspace(min,max,nbins),
          ys=[x_c1,x_c2],
          keys=["censored", "uncensored"],
          title=f"Histogramm",
          xname="risk")
wandb.log({"custom":table})
#KM

#stepfunction
def stepfunc(x,y,eps=1e-4):
    x = np.stack((x-eps,x),axis=1).flatten()
    y = np.stack((y,y),axis=1).flatten()
    return x[1:].copy(),y[:-1].copy()








for idx,threshold in enumerate(thresholds.tolist()): #thresholds
    x_full, y_full = kaplan_meier_estimator(uncensored.numpy(), l_cont_all_val)

    xlow, ylow = kaplan_meier_estimator(uncensored[risk>threshold].numpy(),
                                l_cont_all_val[risk>threshold].numpy())

    xhigh, yhigh = kaplan_meier_estimator(uncensored[risk<=threshold].numpy(),
                                l_cont_all_val[risk<=threshold].numpy())
    
    xfull, yfull = stepfunc(x_full, y_full)
    xlow, ylow =stepfunc(xlow, ylow)
    xhigh, yhigh =stepfunc(xhigh, yhigh)

    #x,y1,y2,y3 = KM_table(xlow,ylow,xhigh,yhigh,xfull,yfull,)
    lineseries = wandb.plot.line_series(
          xs=[xlow,xhigh,xfull],
          ys=[ylow,yhigh,yfull],
          keys=["lowrisk", "highrisk","fullrisk"],
          title=f"KM Stratification at risk={str(round(threshold,2)).replace('.',',')}",
          xname="time")
    
    wandb.log({f"KM_{idx}" :lineseries})
    

# Log the table to wandb
wandb.log({"line_series": table})


wandb.finish()

In [ ]:

def KM_table(xlow,ylow,xhigh,yhigh,xfull,yfull,):
    ### get KM curve on same x values... not needed anymore FML
    low_all = np.stack((xlow,ylow,np.empty_like(ylow)*np.nan,np.empty_like(ylow)*np.nan),1)
    high_all = np.stack((xhigh,np.empty_like(yhigh)*np.nan,yhigh,np.empty_like(yhigh)*np.nan),1)
    full_all = np.stack((xfull,np.empty_like(yfull)*np.nan,np.empty_like(yfull)*np.nan, yfull),1)
    all = np.concatenate((low_all,high_all,full_all),axis=0)

    all = all[all[:, 0].argsort()] # sort by first element 

    for i in range(1,len(all)):
        a = all[i-1,1:] # =:prev-line
        b = all[i,1:]  # =:line
        all[i,1:] = np.where(np.isnan(b),a,b)
        if all[i,0]==all[i-1,0]: #if on same x give info to line, set x of prev-line to nan
            all[i-1,0] = np.nan
            
    all = all[all[:, 0].argsort()]
    all = all[~np.isnan(all[:,0])]
    return all[:,0],all[:,1],all[:,2],all[:,3]  # x, y_low,y_high,y_full



## Final FUnctions for Utils 

In [ ]:


def KM_wandb(run,out,c,event_cond,n_thresholds = 4,nbins = 30):
    print("Start Logging KM-Estimators")
    risk = get_risk(out)
    
    #thresholds
    min,max = risk.min(),risk.max()
    thresholds = np.linspace(min,max,n_thresholds+2)[1:-1]
    
    #hist
    censored = c.type(torch.bool)
    uncensored = ~c.type(torch.bool)
    
    hist_censored = torch.histc(risk[censored],bins=nbins,min = min , max =max ) 
    hist_uncensored = torch.histc(risk[uncensored],bins=nbins,min = min , max =max ) 
    
    table = run.plot.line_series(
          xs =np.linspace(min,max,nbins),
          ys=[hist_censored,hist_uncensored],
          keys=["censored", "uncensored"],
          title=f"Histogramm",
          xname="risk")
    run.log({"risk_histogramm":table})
    
    #KaplanMeier Plots
    x_full, y_full = kaplan_meier_estimator(uncensored.numpy(), event_cond)
    xfull, yfull = stepfunc(x_full, y_full)
    for idx,threshold in enumerate(thresholds): 
        xlow, ylow = kaplan_meier_estimator(uncensored[risk>threshold].numpy(),
                                    event_cond[risk>threshold])

        xhigh, yhigh = kaplan_meier_estimator(uncensored[risk<=threshold].numpy(),
                                    event_cond[risk<=threshold])
        
        xlow, ylow =stepfunc(xlow, ylow)
        xhigh, yhigh =stepfunc(xhigh, yhigh)

        
        lineseries = run.plot.line_series(
            xs=[xlow,xhigh,xfull],
            ys=[ylow,yhigh,yfull],
            keys=["lowrisk", "highrisk","fullrisk"],
            title=f"KM Stratification at risk={str(round(threshold,2)).replace('.',',')}",
            xname="time")
        
        run.log({f"KM_{idx}" :lineseries})
    print("Finished logging KM-Estimators")
    
def get_risk(out):
    h = nn.Sigmoid()(out)
    S = torch.cumprod(1-h,dim = -1)
    risk = -S.sum(dim=1)
    return risk

def stepfunc(x,y,eps=1e-4):
    x = np.stack((x-eps,x),axis=1).flatten()
    y = np.stack((y,y),axis=1).flatten()
    return x[1:],y[:-1]

def do_table(x,y,label):
    return [[x[i],y[i],label] for i in range(len(x))]

In [ ]:
import wandb
import matplotlib.pyplot as plt 
wandb.init(project="MultiModal",entity="tobias-seibel",name="tests")
out_all_val = out_all_val
n_thresholds = 4
nbins = 30
c_all_val = c_all_val
l_cont_all_val = l_cont_all_val 

#risk
def get_risk(out):
    h = nn.Sigmoid()(out)
    S = torch.cumprod(1-h,dim = -1)
    risk = -S.sum(dim=1)
    return risk
risk_all = get_risk(out_all_val)

#thresholds
min,max = risk_all.min(),risk_all.max()
thresholds = torch.linspace(min,max,n_thresholds+2)[1:-1]

#hist 
censored = c_all_val.type(torch.bool)
uncensored = ~c_all_val.type(torch.bool)

x_c1 = torch.histc(risk_all[censored],bins=nbins,min = min , max =max ).numpy()
x1_label = np.chararray(np.shape(x_c1))
x1_label[:]="censored"
x_c2 = torch.histc(risk_all[uncensored],bins=nbins,min = min , max =max ).numpy() 
x2_label = np.chararray(np.shape(x_c2))
x2_label[:]="uncensored"

x1 = np.stack((np.linspace(min,max,nbins),x_c1,x1_label))
x2 = np.stack((np.linspace(min,max,nbins),x_c2,x2_label))
np.concatenate((x1,x2),dim=1).T

table = wandb.Table(
          data = np.concatenate((x1,x2),dim=1).T,
          columns=["bins", "risk","category"],
          
          

          )
wandb.log({"custom":table})
#KM

wandb.finish()


## Better version for custom plots! 

In [ ]:

x = np.linspace(min,max,nbins)
x_c1 = torch.histc(risk_all[censored],bins=nbins,min = min , max =max ).numpy()
x_c2 = torch.histc(risk_all[uncensored],bins=nbins,min = min , max =max ).numpy() 

table_hist = wandb.Table(
          data = do_table(x,x_c1,"censored")+do_table(x,x_c2,"uncensored"),
          columns=["risk", "count","category"],
          )

fields_hist = {"x":"risk","y":"count","groupKeys":"category","title":"Risk Distribution"}
custom_histogram = wandb.plot_table(vega_spec_name="tobias-seibel/risk_distribution",
              data_table=table_hist,
              fields = fields_hist )
              
              
event_cond = l_cont_all_val
threshold = -2
xfull, yfull = kaplan_meier_estimator(uncensored.numpy(), event_cond)

xlow, ylow = kaplan_meier_estimator(uncensored[risk>threshold].numpy(),
                                    event_cond[risk>threshold])

xhigh, yhigh = kaplan_meier_estimator(uncensored[risk<=threshold].numpy(),
                                    event_cond[risk<=threshold])
        


table_KM = wandb.Table(
          data = do_table(xlow,ylow,"low risk group")+do_table(xhigh,yhigh,"risk high group")+do_table(xfull,yfull,"total group"),
          columns=["time","Survival Probability","Group"],)

field_KM = {"x":"time","y":"Survival Probability","groupKeys":"Group"}
custom_KM = wandb.plot_table(vega_spec_name="tobias-seibel/kaplanmeier",
              data_table=table_KM,
              fields = field_KM, 
              string_fields={"title":f"KM Risk Stratification at {round(threshold,2)}"},
              )

wandb.init()

idx = 1 
wandb.log({"Risk Distribution":custom_histogram,"KM{idx}":custom_KM}) #tobias-seibel/risk_distribution # tobias-seibel/kaplanmeier
wandb.finish()


In [ ]:
def KM_wandb(run,out,c,event_cond,n_thresholds = 4,nbins = 30):
    print("Start Logging KM-Estimators")
    risk = get_risk(out)
    
    #thresholds
    min,max = risk.min(),risk.max()
    thresholds = np.linspace(min,max,n_thresholds+2)[1:-1]
    
    #hist
    censored = c.type(torch.bool)
    uncensored = ~c.type(torch.bool)
    
    ###wandb histogram
    x = np.linspace(min,max,nbins)
    x_c1 = torch.histc(risk_all[censored],bins=nbins,min = min , max =max ).numpy()
    x_c2 = torch.histc(risk_all[uncensored],bins=nbins,min = min , max =max ).numpy() 

    table_hist = wandb.Table(
            data = do_table(x,x_c1,"censored")+do_table(x,x_c2,"uncensored"),
            columns=["risk", "count","category"],
            )

    fields_hist = {"x":"risk","y":"count","groupKeys":"category","title":"Risk Distribution"}
    custom_histogram = wandb.plot_table(vega_spec_name="tobias-seibel/risk_distribution",
                data_table=table_hist,
                fields = fields_hist )
    
    wandb.log({"Risk Distribution":custom_histogram})
    ###
    
    
    
    
    #KaplanMeier Plots
    xfull, yfull = kaplan_meier_estimator(uncensored.numpy(), event_cond)
    
    for idx,threshold in enumerate(thresholds): 
        xlow, ylow = kaplan_meier_estimator(uncensored[risk>threshold].numpy(),
                                    event_cond[risk>threshold])

        xhigh, yhigh = kaplan_meier_estimator(uncensored[risk<=threshold].numpy(),
                                    event_cond[risk<=threshold])
        
        
        table_KM = wandb.Table(data = do_table(xlow,ylow,"low risk group")+do_table(xhigh,yhigh,"risk high group")+do_table(xfull,yfull,"total group"),
                        columns=["time","Survival Probability","Group"],)

        field_KM = {"x":"time","y":"Survival Probability","groupKeys":"Group"}
        custom_KM = wandb.plot_table(vega_spec_name="tobias-seibel/kaplanmeier",
                    data_table=table_KM,
                    fields = field_KM, 
                    string_fields={"title":f"KM Risk Stratification at {round(threshold,2)}"},
                    )
        run.log({f"KM_{idx}" :custom_KM})
    print("Finished logging KM-Estimators")
    

In [ ]:
run = wandb.init()
KM_wandb(run,out_all_val,c_all_val,l_cont_all_val,n_thresholds = 4,nbins = 30)
run.finish()

# Transformer 


In [1]:
import torch
from torch import nn
from utils import *
class Classifier_Head(nn.Module):
    def __init__(self,outsize,d_hidden=256,t_bins=4):
        super(Classifier_Head,self).__init__()

        self.linear1 = nn.Linear(outsize,d_hidden)
        torch.nn.init.kaiming_normal_(self.linear1.weight)
        self.activ1 = nn.ReLU()
        self.linear2  = nn.Linear(d_hidden,d_hidden)
        torch.nn.init.kaiming_normal_(self.linear2.weight)
        self.activ2 = nn.ReLU()
        self.fc = nn.Linear(d_hidden,t_bins) # TODO test add layer
    def forward(self,x):
        x = torch.flatten(x,start_dim=1)
        x = self.activ1(self.linear1(x))
        x = self.activ2(self.linear2(x))
        return self.fc(x)
    
    
class TransformerMil_Survival(nn.Module):
  def __init__(self,d_seq,d_transformer,bins):
    super(TransformerMil_Survival,self).__init__()
    d_out = d_hidden = 256
    self.lin_embedder1 = nn.Linear(d_seq,d_transformer,dropout)
    #self.Encoder = torch.nn.TransformerEncoder(nn.TransformerEncoderLayer(d_transformer,
    #                                                                 nhead=2,dropout=0.1,activation=nn.GELU(),batch_first=True)
    #                                      ,num_layers=2)
    self.Encoder=nn.TransformerEncoderLayer(d_transformer,nhead=2,dropout=dropout,activation=nn.GELU(),batch_first=True)
    self.lin_embedder2 = nn.Linear(d_transformer,d_out)
    self.Classifier_Head = Classifier_Head(outsize = d_out,d_hidden=d_hidden,t_bins=bins)
  def forward(self,x):
    
    x = self.lin_embedder1(x)
    x = self.Encoder(x)
    out = x.mean(dim=-2)
    out = self.lin_embedder2(out)
    return self.Classifier_Head(out)


/home/seibel/.local/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda"
bins = 4
model = TransformerMil_Survival(d_seq=2048,d_transformer=1024,bins=bins)
model.to(device)


criterion = Survival_Loss(0.25) 
optimizer = torch.optim.Adam(model.parameters(),lr=0.00003,betas=[0.9,0.999],weight_decay=1e-5,)

In [3]:
nseq = 15000
B=1
dims = 2048

x = torch.rand(size=(B,nseq,dims)).to(device)
l = torch.randint(low=0, high=bins, size=(B,)).to(device)
c = torch.randint(low=0, high=2, size=(B,)).to(device)  ## Groundtruth boolean information wether the the patient is censored  c = 1 or uncensored  

out = model(x)
loss = criterion(out,c,l)
loss.backward()

RuntimeError: CUDA out of memory. Tried to allocate 1.68 GiB (GPU 0; 10.92 GiB total capacity; 10.02 GiB already allocated; 60.69 MiB free; 10.06 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF